In [4]:
!pip install langchain langchain-openai langchain-community langchain-core openai pypdf azure-keyvault-secrets azure-identity faiss-cpu dotenv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
           ---------------------------------------- 0.0/145.4 kB ? eta -:--:--
           ---------- -------------------------- 41.0/145.4 kB 991.0 kB/s eta 0:00:01
           -------------------------------------- 145.4/145.4 kB 2.2 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— Preparing metadata (pyproject.toml) did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [6 lines of output]
            usage: setup.py [global

In [2]:
import json
import os
from azure.identity import DefaultAzureCredential
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import LLMChain
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from openai import AzureOpenAI

deployment_name = "uptale-gpt-35-turbo"

llm = AzureChatOpenAI(
    azure_deployment=deployment_name
)

In [7]:
def process_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    return texts

In [13]:
def generate_title(texts):
    combined_text = " ".join(text.page_content for text in texts)
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Create a catchy title for a 360-degree educational experience based on it."
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(document=combined_text)
    return response.strip()

In [9]:
def generate_scenes(texts, num_scenes=5):
    combined_text = " ".join(text.page_content for text in texts)
    scenes = []
    for i in range(num_scenes):
        prompt = PromptTemplate(
            input_variables=["scene_number", "document"],
            template="""
            Create scene {scene_number} for a 360-degree educational experience based on the document content.
            Include:
            1. A brief description of the scene
            2. Educational content relevant to the document
            3. A transition to the next scene (use one of the following):
               - Tag Door => Target Scene
               - Tag QCM => Go to Scene (with up to 4 multiple-choice answers)
               - Tag Microphone Rules => Allow to trigger an action to go to another scene

            Format the output as a JSON object.
            """
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        scene = chain.run(scene_number=i+1, document=combined_text)
        try:
            scenes.append(json.loads(scene))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for scene {i+1}. Skipping this scene.")
    return scenes

In [10]:
def generate_360_experience(pdf_path):
    texts = process_pdf(pdf_path)
    title = generate_title(texts)
    scenes = generate_scenes(texts)

    experience = {
        "title": title,
        "scenes": scenes
    }

    return json.dumps(experience, indent=2)

In [14]:
pdf_path = "./content/example1.pdf"
experience_json = generate_360_experience(pdf_path)
print(experience_json)

# Optionally, save the JSON to a file
with open("./content/360_experience.json", "w") as f:
    f.write(experience_json)

{
  "title": "\"Revolutionizing Education: A 360-Degree Journey towards Limitless Learning\"",
  "scenes": [
    {
      "scene1": {
        "description": "Scene 1 takes place in a virtual classroom. The room is equipped with interactive whiteboards, desks with tablets, and a teacher's desk at the front.",
        "educationalContent": "The document discusses the benefits of using technology in education. It highlights how interactive whiteboards and tablets can enhance student engagement and learning outcomes.",
        "transition": {
          "tag": "Tag Door",
          "targetScene": "scene2"
        }
      }
    },
    {
      "scene2": {
        "description": "Scene 2: The Rainforest",
        "educationalContent": "In this scene, the user is transported to a lush rainforest. They are surrounded by tall trees, vibrant flowers, and the sound of chirping birds. The immersive experience allows the user to explore the different layers of the rainforest ecosystem and learn about 